# Code Conversion using Ollama Inference Models

In [1]:
from itertools import permutations
from constants import *

import pandas as pd
import numpy as np
import ollama
import os
import re

In [2]:
RESULTS_FOLDER = "results"
GENERATED_CODE_FOLDER = os.path.join(RESULTS_FOLDER, "generated_code")
MODEL = "codegemma2b"
TASK = "bit_ops"
LANG2LANG = list(permutations(LANGS.keys(), 2))

In [3]:
def get_prompt(system_prompt, user_prompt):
    prompt = []
    if system_prompt:
        prompt.append({"role": "system", "content": system_prompt})
    prompt.append({"role": "user", "content": user_prompt})
    return prompt

In [4]:
prompt = "Convert the following code from {} to {}. This is the requirement for the code - {}\n```{}```\n"
pattern = r"(?s)```(?:\w+\s*)?\n?(.*?)```"

In [5]:
ollama.pull(OLLAMA_TAGS[MODEL])

{'status': 'success'}

In [6]:
for idx, (lang1, lang2) in enumerate(LANG2LANG):
    formatted_prompt = prompt.format(LANGS[lang1], LANGS[lang2], TASK_DESCRIPTION[TASK], REFERENCE_CODE[lang1][TASK])
    if OLLAMA_SYSTEM_PROMPT_SUPPORT[MODEL]:
        chat_prompt = get_prompt("You are a helpful code conversion assistant.", formatted_prompt)
    else:
        chat_prompt = get_prompt(None, formatted_prompt)
    try:
        rsp = ollama.chat(OLLAMA_TAGS[MODEL], messages=chat_prompt, options={"seed": 42, "timeout": 60})
    except:
        rsp = ollama.chat(OLLAMA_TAGS[MODEL], messages=chat_prompt, options={"seed": 41, "timeout": 60})
    try:
        code = re.findall(pattern, rsp["message"]["content"])[0]
        with open(os.path.join(GENERATED_CODE_FOLDER, f"{MODEL}-{TASK}-{lang1.lower()}-{lang2.lower()}.{lang2.lower()}"), "w") as fp:
            fp.write(code + "\n")
        print(f"completed {MODEL} {TASK} {lang1} {lang2}")
    except:
        print(f"error in {MODEL} {TASK} {lang1} {lang2}")
        with open(os.path.join(GENERATED_CODE_FOLDER, f"{MODEL}-{TASK}-{lang1.lower()}-{lang2.lower()}.md"), "w") as fp:
            fp.write(rsp["message"]["content"] + "\n")

completed codegemma2b bit_ops C CPP
error in codegemma2b bit_ops C GO
completed codegemma2b bit_ops C JAVA
error in codegemma2b bit_ops C JS
error in codegemma2b bit_ops C PY
completed codegemma2b bit_ops C RS
error in codegemma2b bit_ops CPP C
error in codegemma2b bit_ops CPP GO
error in codegemma2b bit_ops CPP JAVA
error in codegemma2b bit_ops CPP JS
error in codegemma2b bit_ops CPP PY
error in codegemma2b bit_ops CPP RS
completed codegemma2b bit_ops GO C
error in codegemma2b bit_ops GO CPP
completed codegemma2b bit_ops GO JAVA
completed codegemma2b bit_ops GO JS
completed codegemma2b bit_ops GO PY
error in codegemma2b bit_ops GO RS
completed codegemma2b bit_ops JAVA C
completed codegemma2b bit_ops JAVA CPP
completed codegemma2b bit_ops JAVA GO
completed codegemma2b bit_ops JAVA JS
completed codegemma2b bit_ops JAVA PY
completed codegemma2b bit_ops JAVA RS
completed codegemma2b bit_ops JS C
completed codegemma2b bit_ops JS CPP
completed codegemma2b bit_ops JS GO
completed codegemma2b